In [76]:
import pandas as pd
import numpy as np
import pandas as pd
import itertools 
import matplotlib.pyplot as plt
from terminaltables import AsciiTable
from scipy.stats import chi2

In [77]:
#SE IMPORTA EL CSV
ford_data = pd.read_csv('../CSV/Ford2.csv')
ford_data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-04-01,4.63,4.77,4.36,4.40,3.736051,119174500
1,2020-04-02,4.39,4.57,4.25,4.36,3.702087,90891700
2,2020-04-03,4.40,4.42,4.17,4.24,3.600194,85659900
3,2020-04-06,4.54,4.67,4.41,4.53,3.846434,102205300
4,2020-04-07,4.85,5.03,4.63,4.71,3.999273,115150400
...,...,...,...,...,...,...,...
980,2024-02-23,12.10,12.26,12.09,12.14,12.140000,32152100
981,2024-02-26,12.13,12.27,11.95,11.95,11.950000,40241300
982,2024-02-27,12.02,12.17,11.96,12.00,12.000000,36508800
983,2024-02-28,11.97,12.40,11.96,12.30,12.300000,51447900


In [78]:
#SE BORRAN LAS COLUMNAS QUE NO SE USARAN
ford_data = ford_data.drop(columns=['Open', 'High', 'Low', 'Close','Volume'])
ford_data

,Date,Adj Close
0,2020-04-01,3.736051
1,2020-04-02,3.702087
2,2020-04-03,3.600194
3,2020-04-06,3.846434
4,2020-04-07,3.999273
...,...,...
980,2024-02-23,12.140000
981,2024-02-26,11.950000
982,2024-02-27,12.000000
983,2024-02-28,12.300000


In [79]:
#SE CALCULA EL CAMBIO PORCENTUAL DE CADA REGISTRO DE ACUERDO AL REGISTRO ANTERIOR
percentage = [0,]
for i in range(1, len(ford_data)):
    percent = (ford_data['Adj Close'][i] - ford_data['Adj Close'][i-1])/ford_data['Adj Close'][i-1]
    percentage.append(percent)
ford_data.insert(2, 'Change %', percentage)

#SE CALCULA LA DESVIACION ESTANDAR DE LOS 20 DATOS ANTERIORES
std_dev = [0,]*20
for i in range(21, len(percentage) + 1):
    standar_dev = np.std(percentage[i-20:i])
    std_dev.append(standar_dev)
ford_data.insert(3, 'std dev', std_dev)

#SE ASIGNA EL ESTADO DE ACUERDO A LA DESVIACION ESTANDAR DE LAS 20 SEMANAS ANTERIORES
states = [0,]*20
for j in range(20, len(percentage)):
    change = percentage[j]
    desv = std_dev[j]
    if change <= -2*desv:
        state = "B3"
    elif -2*desv < change <= -desv:
        state = "B2"
    elif -desv < change < 0:
        state = "B1"
    elif 0 <= change < desv:
        state = "S1"
    elif desv <= change < 2*desv:
        state = "S2"
    elif 2*desv <= change:  
        state = "S3"
    states.append(state)
ford_data.insert(4, 'Current State', states)

#SE ASIGNA UNA NUEVA COLUMNA CON EL ESTADO SIGUIENTE AL REGISTRO
next_state_column=[]
for i in range(len(ford_data)-1):
    next_state_column.append(ford_data['Current State'][i+1])

previows_state_column=[0]
for i in range(1,len(ford_data)-1):
    previows_state_column.append(ford_data['Current State'][i-1])
    
#SE BORRA LA ULTIMA FILA POR QUE NO TIENE ESTADO SIGUIENTE
ford_data = ford_data.drop(ford_data.index[-1])
ford_data.insert(5, 'Next State', next_state_column)
ford_data.insert(4, 'Previous State', previows_state_column)

complete_table = AsciiTable([ford_data.columns]+list(ford_data.values))
print("Complete Table \n",complete_table.table)


Complete Table 
 +------------+-----------+------------------------+----------------------+----------------+---------------+------------+
| Date       | Adj Close | Change %               | std dev              | Previous State | Current State | Next State |
+------------+-----------+------------------------+----------------------+----------------+---------------+------------+
| 2020-04-01 | 3.736051  | 0.0                    | 0.0                  | 0              | 0             | 0          |
| 2020-04-02 | 3.702087  | -0.009090882324679096  | 0.0                  | 0              | 0             | 0          |
| 2020-04-03 | 3.600194  | -0.027523124118909146  | 0.0                  | 0              | 0             | 0          |
| 2020-04-06 | 3.846434  | 0.06839631419862369    | 0.0                  | 0              | 0             | 0          |
| 2020-04-07 | 3.999273  | 0.03973524568470437    | 0.0                  | 0              | 0             | 0          |
| 2020-04-08 | 

In [86]:
def calcular_precios_acciones(data):
    # Inicializar una lista vacía para almacenar los precios calculado
    precios_acciones = []

    # Recorrer la tabla de datos
    for i in range(len(data)):
        if i == 0:
            # Para el primer día, el precio es simplemente el Adj Close
            precio_actual = data['Adj Close'][i]
        else:
            # Para los días siguientes, calcular el precio usando el cambio porcentual y el precio del día anterior
            cambio_porcentual = data['Change %'][i]
            precio_anterior = data['Adj Close'][i-1]
            precio_actual = precio_anterior * (1 + cambio_porcentual)
        
        # Agregar el precio calculado a la lista
        precios_acciones.append(precio_actual)

    return precios_acciones

# Utilizar la función para calcular los precios de las acciones
precios_calculados = calcular_precios_acciones(ford_data)

# Imprimir los precios calculados
for i, precio in enumerate(precios_calculados, start=1):
    print(f'Día {i}: {precio}')


Día 1: 3.736051
Día 2: 3.702087
Día 3: 3.600194
Día 4: 3.846434
Día 5: 3.9992730000000005
Día 6: 4.270986
Día 7: 4.55968
Día 8: 4.381369
Día 9: 4.491753000000001
Día 10: 4.270986
Día 11: 4.194566
Día 12: 4.347404
Día 13: 4.22853
Día 14: 4.050219
Día 15: 4.050219
Día 16: 4.152111
Día 17: 4.135129
Día 18: 4.38986
Día 19: 4.568171
Día 20: 4.466279
Día 21: 4.321932
Día 22: 4.177584
Día 23: 4.126638
Día 24: 4.220039
Día 25: 4.135129
Día 26: 4.135129
Día 27: 4.449296
Día 28: 4.347404
Día 29: 4.22853
Día 30: 4.007763
Día 31: 4.152111
Día 32: 4.160602
Día 33: 4.508734
Día 34: 4.500244
Día 35: 4.661572
Día 36: 4.780447
Día 37: 4.797429
Día 38: 4.958759
Día 39: 5.120088
Día 40: 4.967249
Día 41: 4.848376
Día 42: 4.984230999999999
Día 43: 5.009705
Día 44: 5.255944
Día 45: 5.578603
Día 46: 6.232412
Día 47: 6.393742
Día 48: 6.147502
Día 49: 5.782388
Día 50: 5.204998
Día 51: 5.485201
Día 52: 5.519166
Día 53: 5.561621000000001
Día 54: 5.374818
Día 55: 5.374818
Día 56: 5.289907
Día 57: 5.33236299999999

In [81]:
#SE QUITAN LAS PRIMERAS 20 FILAS, PUES ESTAS NO CUENTAN CON ESTADO ACTUAL
ford_trim=ford_data.iloc[21:]
table_week_21 = AsciiTable([ford_trim.columns]+list(ford_trim.values))
print("Table since week 21 \n",table_week_21.table)

Table since week 21 
 +------------+-----------+------------------------+----------------------+----------------+---------------+------------+
| Date       | Adj Close | Change %               | std dev              | Previous State | Current State | Next State |
+------------+-----------+------------------------+----------------------+----------------+---------------+------------+
| 2020-05-01 | 4.177584  | -0.03339895213529503   | 0.040472763891518396 | B1             | B1            | B1         |
| 2020-05-04 | 4.126638  | -0.012195086921053077  | 0.03995608107590788  | B1             | B1            | S1         |
| 2020-05-05 | 4.220039  | 0.022633679038481222   | 0.037655086431775536 | B1             | S1            | B1         |
| 2020-05-06 | 4.135129  | -0.020120667131275284  | 0.03717865556646528  | S1             | B1            | S1         |
| 2020-05-07 | 4.135129  | 0.0                    | 0.03399871069865621  | B1             | S1            | S3         |
| 2020-05-

In [82]:
#FUNCIONES PARA SOLUCIONAR DIVISIONES POR 0
def resizeExpectedValues(freq_matrix, states):
    index_to_delete = []
    for index in range(len(freq_matrix)):
        if 0 in freq_matrix[index]:
            if index > 0:
                freq_matrix[index-1] += freq_matrix[index]
                freq_matrix[:,index-1] += freq_matrix[:,index] 
            else:
                freq_matrix[index+1] += freq_matrix[index]
                freq_matrix[:,index+1] += freq_matrix[:,index] 
            index_to_delete.append(index)
    freq_matrix = np.delete(freq_matrix, index_to_delete, axis=0)
    freq_matrix = np.delete(freq_matrix, index_to_delete, axis=1)
    
    for i in range(len(states)):
        if i in index_to_delete:
            if index > 0:
                states[i-1]=f"{states[i-1]}+{states[i]}"
            else:
                states[i+1]=f"{states[i]}+{states[i+1]}"
    states=np.delete(states,index_to_delete)
    return freq_matrix, states

In [83]:
def validate_markov_rule(data):

    #PROBABILIDAD ESPERADA
    states=np.unique(data['Current State'])
    states_mapping = {}
    for i,state in enumerate(states):
        states_mapping[state]=i

    #SE CREA LA MATRIZ DE FRECUENCIAS ESPERADAS
    freq_expected=np.zeros(((len(states_mapping)),len(states_mapping)))
    for i in range(len(data)):
        freq_expected[states_mapping[data.iloc[i]['Current State']], states_mapping[data.iloc[i]['Next State']]] += 1

    #FUNCION PARA FRECUENCIAS DE 0
    if 0 in freq_expected:
        freq_expected, states=resizeExpectedValues(freq_expected,states)
        states_mapping={new_state: i for i,new_state in enumerate(states)}

    #SE DETERMINA LA MATRIZ DE PROBABILIDAD ESPERADA
    transition_expected = np.zeros(((len(states)), len(states)))
    for i in range(len(freq_expected)):
        transition_expected[i] = freq_expected[i] / sum(freq_expected[i])

    data_frame_expected = pd.DataFrame(transition_expected, columns=states, index=states)
    print(f"Probabilidad Esperada\n{data_frame_expected}\n")
    transition_expected=np.tile(transition_expected,(len(transition_expected),1))
    
    #PROBABILIDAD OBSERVADA
    dict_states_before = {prev_state :{curr_state: {next_state: 0 for next_state in states} for curr_state in states} for prev_state in states}
    states_before = list(itertools.product(states,repeat=2))
    states_list_before = []
    for i,state_before in enumerate(states_before):
        complete_state=f"{state_before[0]} - {state_before[1]}"
        states_list_before.append(complete_state)
    
    #SE CREA LA MATRIZ DE FRECUENCIAS OBSERVADA
    for i in range(len(data)):
        row_in_data=data.iloc[i]
        prev_state = row_in_data['Previous State']
        curr_state = row_in_data['Current State']
        next_state = row_in_data['Next State']
        for _,prev_state_map in enumerate(states):
            if prev_state in prev_state_map:
                for _,curr_state_map in enumerate(states):
                    if curr_state in curr_state_map:
                        for _,next_state_map in enumerate(states):
                            if next_state in next_state_map:
                                dict_states_before[prev_state_map][curr_state_map][next_state_map] += 1

    freq_observed=np.zeros(((len(states_list_before)),len(states)))
    for i,state_i in enumerate(states):
        for j,state_j in enumerate(states):
            for z, state_z, in enumerate(states):
                coordinate_y=j+i*len(dict_states_before)
                freq_observed[coordinate_y,z]=dict_states_before[state_i][state_j][state_z]

    #SE DETERMINA LA MATRIZ DE PROBABILIDAD OBSERVADA
    transitions_observed=np.zeros(((len(freq_observed)),len(states)))
    for i in range(len(freq_observed)):
        transitions_observed[i]=freq_observed[i]/sum(freq_observed[i])

    data_frame_observed=pd.DataFrame(transitions_observed,columns=states,index=states_list_before)
    print(f"Probabilidad Observada \n {data_frame_observed}\n")

    #CHI CUADRADO
    chi2_observed = (np.square(transitions_observed - transition_expected)) / transition_expected
    for i in range(len(chi2_observed)):
        chi2_observed[i]=chi2_observed[i]*sum(freq_observed[i])
    chi2_observed=np.sum(chi2_observed)

    #VALORES NECESARIOS
    rows,columns=transitions_observed.shape
    grades=(rows-1)*(columns-1)
    alpha=0.05
    chi2_theory=chi2.ppf(1-alpha,grades)
    accepted=chi2_observed<chi2_theory

    print(f"Número de Filas: {rows}")
    print(f"Número de Columnas: {columns}")
    print(f"Grados de Libertad: {grades}")
    print(f"Porcentaje de Confianza: {(1-alpha)*100}%")
    print(f"Chi Cuadrado: {chi2_observed}")
    print(f"Chi Teórico: {chi2_theory}")
    print(f"¿Se Acepta?: {accepted}")
    return accepted

is_accepted=validate_markov_rule(ford_trim[['Previous State', 'Current State', 'Next State']])

Probabilidad Esperada
             B1     B2+B3        S1     S2+S3
B1     0.371014  0.118841  0.362319  0.147826
B2+B3  0.322581  0.177419  0.290323  0.209677
S1     0.349854  0.128280  0.376093  0.145773
S2+S3  0.370861  0.112583  0.357616  0.158940

Probabilidad Observada 
                      B1     B2+B3        S1     S2+S3
B1 - B1        0.286822  0.178295  0.364341  0.170543
B1 - B2+B3     0.219512  0.195122  0.341463  0.243902
B1 - S1        0.320000  0.136000  0.432000  0.112000
B1 - S2+S3     0.352941  0.098039  0.333333  0.215686
B2+B3 - B1     0.250000  0.100000  0.400000  0.250000
B2+B3 - B2+B3  0.363636  0.181818  0.227273  0.227273
B2+B3 - S1     0.388889  0.111111  0.361111  0.138889
B2+B3 - S2+S3  0.269231  0.115385  0.538462  0.076923
S1 - B1        0.433333  0.075000  0.416667  0.075000
S1 - B2+B3     0.386364  0.181818  0.272727  0.159091
S1 - S1        0.333333  0.131783  0.372093  0.162791
S1 - S2+S3     0.360000  0.140000  0.340000  0.160000
S2+S3 - B1     0.517

In [84]:
def markov_model(data):
    #PROBABILIDAD OBSERVADA
    states=np.unique(data['Current State'])
    states_mapping = {}
    for i,state in enumerate(states):
        states_mapping[state]=i

    #PROBABILIDAD ESPERADA
    transition_matrix = np.zeros(((len(states_mapping)), len(states_mapping)))
    for i in range(len(data)):
        transition_matrix[states_mapping[data.iloc[i]['Current State']], states_mapping[data.iloc[i]['Next State']]] += 1
    for i in range(len(transition_matrix)):
        transition_matrix[i] = transition_matrix[i] / sum(transition_matrix[i])
    data_frame_expected = pd.DataFrame(transition_matrix, columns=list(states_mapping.keys()), index=list(states_mapping.keys()))
    print(f"Probabilidad Esperada\n{data_frame_expected}\n")

    return transition_matrix
#SE ENVIAN LAS COLUMNAS DE ESTADO ACTUAL Y SIGUIENTE
transition_matrix = markov_model(ford_trim[['Current State', 'Next State']])

Probabilidad Esperada
          B1        B2        B3        S1        S2        S3
B1  0.371014  0.104348  0.014493  0.362319  0.113043  0.034783
B2  0.284314  0.117647  0.068627  0.333333  0.127451  0.068627
B3  0.500000  0.090909  0.045455  0.090909  0.272727  0.000000
S1  0.349854  0.107872  0.020408  0.376093  0.119534  0.026239
S2  0.380531  0.088496  0.017699  0.362832  0.088496  0.061947
S3  0.342105  0.131579  0.000000  0.342105  0.105263  0.078947



In [85]:
def sendInfoToAnalize():
    return is_accepted, transition_matrix